# CPUとGPUの比較

これから機械学習において、CPUとGPUにどのような違いが出るかを体験していただきます。

## 準備

In [ ]:
# 必要なライブラリのインストール
!pip install -r requirement.txt

## 行列の内積計算
CPUとGPUで行列の内積計算の速度を比較します。

### 準備

In [ ]:
# 必要なライブラリのインポート
import numpy as np
import torch
import time

In [ ]:
# 行列のサイズを設定（大きいほど計算が重くなり、差がわかりやすくなります）
matrix_size = 8000

### 実行

<b>CPU</b>

In [ ]:
# CPUでの計算
print("CPU計算開始...")
A_cpu = np.random.rand(matrix_size, matrix_size)
B_cpu = np.random.rand(matrix_size, matrix_size)
start_time = time.time()
C_cpu = np.dot(A_cpu, B_cpu)
cpu_time = time.time() - start_time
print(f"CPUでの計算時間: {cpu_time:.4f} 秒")

<b>GPU</b>

In [ ]:
print("GPU計算開始...")
A_gpu = torch.rand(matrix_size, matrix_size, device='cuda')
B_gpu = torch.rand(matrix_size, matrix_size, device='cuda')
start_time = time.time()
C_gpu = torch.mm(A_gpu, B_gpu)
gpu_time = time.time() - start_time
print(f"GPUでの計算時間: {gpu_time:.4f} 秒")

GPUでは、数千の小さなコアを持っており、同時に大量の計算をすることがかのうであるため、大規模な行列の計算などを高速に処理することが可能となります。

## 機械学習
ディープラーニングでは大規模な行列同士の積が必要となります。<br>
そのため、GPUの並列処理能力が大きな役割を果たします。

今回は学習データ<b>300枚</b>、テストデータ<b>100枚</b>を用いて、<b>1回</b>(1エポック)訓練を行います。<br>
通常は<b>何千、何万以上</b>のデータを用いて<b>何百回</b>も繰り返し訓練し、精度を上げていきます。（タスクやモデルにもよりますが）

### 準備

In [ ]:
# 使用するライブラリのインポート
import math
import tqdm
import time
import torch
import numpy as np
from torch import nn
from PIL import Image
import matplotlib.pyplot as plt

from unet import UNet
from dataset import VOCDataset
from torch.utils.data import DataLoader

In [ ]:
# VOC2012で用いるラベル
CLASSES = ['backgrounds','aeroplane','bicycle','bird','boat','bottle',
            'bus','car' ,'cat','chair','cow', 
            'diningtable','dog','horse','motorbike','person', 
            'potted plant', 'sheep', 'sofa', 'train', 'monitor','unlabeld'
            ]

# カラーパレットの作成
COLOR_PALETTE = np.array(Image.open("./VOCdevkit/VOC2012_sample/SegmentationClass/2007_000170.png").getpalette()).reshape(-1,3)
COLOR_PALETTE = COLOR_PALETTE.tolist()[:len(CLASSES)]

# 使用データのリストファイル
train_list_path = 'VOCdevkit/VOC2012_sample/listfile/train_list_300.txt'
val_list_path = 'VOCdevkit/VOC2012_sample/listfile/val_list_100.txt'

# データのあるディレクトリ
img_dir = 'VOCdevkit/VOC2012_sample/JPEGImages'
gt_dir = 'VOCdevkit/VOC2012_sample/SegmentationClass'

In [ ]:
# リストファイルの読み込み
with open(train_list_path, 'r') as f:
    train_list = f.read().splitlines()
with open(val_list_path, 'r') as g:
    val_list = g.read().splitlines()

# データセット作成
train_ds = VOCDataset(img_list=train_list, phase='trian', img_dir=img_dir, gt_dir=gt_dir)
val_ds = VOCDataset(img_list=val_list, phase='val', img_dir=img_dir, gt_dir=gt_dir)
print(f"len(train_data): {train_ds.__len__()}")
print(f"len(test_data): {val_ds.__len__()}")

# データローダーの作成
train_dl = DataLoader(train_ds, batch_size=16, num_workers=4, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=16, num_workers=4, shuffle=True)

# その他設定
epochs = 1 # エポック数
model = UNet() # 機械学習モデル
criterion =  nn.CrossEntropyLoss() # 損失関数

# 重みの読み込み。
chkp_path = 'weights/20241105_demo_ep2.pth'

In [ ]:
# 訓練
def train(dataloader, model, optimizer, criterion, device):
    start = time.time()

    model.train()
    size = len(dataloader.dataset)
    batch_size = dataloader.batch_size
    step_total = math.ceil(size/batch_size)
    total_loss = 0.0

    with tqdm.tqdm(enumerate(dataloader), total=step_total) as pbar:
        for batch, item in pbar:
            inp, gt = item[0].to(device), item[1].to(device)

            # 推論
            pred = model(inp)

            # 損失誤差を計算
            loss = criterion(pred, gt)
            total_loss += loss.item()

            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # プログレスバーに損失を表示
            pbar.set_postfix(loss=loss.item())

        avg_loss = total_loss / step_total
        # print(f"Train Loss: {avg_loss:.4f}")
    
    end = time.time()
    return end - start

# テスト
def test(dataloader, model, criterion, device):
    start = time.time()

    model.eval()
    size = len(dataloader.dataset)
    batch_size = dataloader.batch_size
    step_total = math.ceil(size/batch_size)
    total_loss = 0.0

    with torch.no_grad():
        for item in dataloader:
            inp, gt = item[0].to(device), item[1].to(device)
            pred = model(inp)
            loss = criterion(pred, gt)
            total_loss += loss.item()

        avg_loss = total_loss / step_total
        # print(f"Val Loss: {avg_loss:.4f}")
        
    end = time.time()
    return end - start

# 推論結果の可視化
def visualize(model, dataloader):
    model.to('cpu')
    model.softmax = True
    model.eval()

    with torch.no_grad():
        for inp, gt in dataloader:
            inp = inp.to('cpu')
            pred = model(inp)

            gt_ = torch.argmax(gt, dim=1).cpu().numpy()
            pred_ = torch.argmax(pred, dim=1).cpu().numpy()

            k = 0
            inp_np = inp[k].numpy().transpose(1, 2, 0)
            gt_np = gt_[k]
            pred_np = pred_[k]

            img_gt = [[0 for j in range(256)] for i in range(256)]
            img_pred = [[0 for j in range(256)] for i in range(256)]

            for height in range(256):
                for width in range(256):
                    index_1 = gt_np[height,width]
                    index_2 = pred_np[height,width]
                    rgb_1 = COLOR_PALETTE[index_1]
                    rgb_2 = COLOR_PALETTE[index_2]
                    img_gt[height][width] = rgb_1
                    img_pred[height][width] = rgb_2

            img_gt = np.asarray(img_gt)
            img_pred = np.asarray(img_pred)
            inp_np = inp_np.astype(np.uint8)
            break
        
    fig = plt.figure(figsize=(10, 6))
    for i, im in enumerate([inp_np, img_gt, img_pred]):
        fig.add_subplot(2,3,i+1).set_title(str(i))
        plt.imshow(im)
    plt.show()

### 実行

<b>CPU</b>

In [ ]:
device = "cpu"
checkpoint = torch.load(chkp_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'], strict=False)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # 最適化手法

In [ ]:
train_time = train(train_dl, model, optimizer, criterion, device)
test_time = test(val_dl, model, criterion, device)
print(f'学習時間：{train_time}[s]')
print(f'評価時間：{test_time}[s]')

<b>GPU</b>

In [ ]:
device = "cuda"
checkpoint = torch.load(chkp_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'], strict=False)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # 最適化手法

In [ ]:
train_time = train(train_dl, model, optimizer, criterion, device)
test_time = test(val_dl, model, criterion, device)
print(f'学習時間：{train_time}')
print(f'評価時間：{test_time}')

1回（1エポック）の訓練でも大きな違いが出ました。<br>
通常これを何百回と繰り返すため、良いGPUが必要となります。